In [40]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import re, os, csv, pathlib, ast
import pandas as pd
from statistics import mean, variance

accent_short_forms = {"hindi":"HIN", "korean":"KOR", "vietnamese":"VTN", "arabic":"ARB", "chinese":"CHN", "spanish":"ESP"}
accent_map = {"ABA":"arabic","SKA":"arabic","YBAA":"arabic","ZHAA":"arabic",
              "BWC":"chinese","LXC":"chinese","NCC":"chinese","TXHC":"chinese",
              "ASI":"hindi","RRBI":"hindi","SVBI":"hindi","TNI":"hindi",
              "HJK":"korean","HKK":"korean","YDCK":"korean","YKWK":"korean",
              "EBVS":"spanish","ERMS":"spanish","MBMPS":"spanish","NJS":"spanish",
              "HQTV":"vietnamese","PNV":"vietnamese","THV":"vietnamese","TLV":"vietnamese"
              }
raw_string="""|ABA|M|Arabic|1129|150|\n|SKA|F|Arabic|974|150|\n|YBAA|M|Arabic|1130|149|\n|ZHAA|F|Arabic|1132|150|\n|BWC|M|Chinese|1130|150|\n|LXC|F|Chinese|1131|150|\n|NCC|F|Chinese|1131|150|\n|TXHC|M|Chinese|1132|150|\n|ASI|M|Hindi|1131|150|\n|RRBI|M|Hindi|1130|150|\n|SVBI|F|Hindi|1132|150|\n|TNI|F|Hindi|1131|150|\n|HJK|F|Korean|1131|150|\n|HKK|M|Korean|1131|150|\n|YDCK|F|Korean|1131|150|\n|YKWK|M|Korean|1131|150|\n|EBVS|M|Spanish|1007|150|\n|ERMS|M|Spanish|1132|150|\n|MBMPS|F|Spanish|1132|150|\n|NJS|F|Spanish|1131|150|\n|HQTV|M|Vietnamese|1132|150|\n|PNV|F|Vietnamese|1132|150|\n|THV|F|Vietnamese|1132|150|\n|TLV|M|Vietnamese|1132|150|"""
raw_strings=raw_string.split('\n')
gender_map={}
for lne in raw_strings:
    attrs=lne.split('|')
    gender_map[attrs[1]]=attrs[2]

composed_accent_map = {k: accent_short_forms.get(v) for k, v in accent_map.items()}

def replace_with_short_forms(s):
    for key, value in accent_short_forms.items():
        s = s.replace(key, value)
    return s

def group_speakers(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    return str(ret)

def group_accents(s):
    ret = {}
    speaker_counts = ast.literal_eval(s)
    for speaker, count in speaker_counts.items():
        accent = composed_accent_map[speaker]
        if accent not in ret:
            ret[accent] = {}
        ret[accent][speaker] = count
    accent_counts={}
    for accent in ret:
        cnt=0
        for speaker in ret[accent]:
            cnt+=ret[accent][speaker]
        accent_counts[accent]=cnt
    return str(accent_counts)

def last_name(pth):
    return pathlib.PurePath(pth).name

def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]

def get_each_run(lne):
    return list(map(float, re.findall(': (.+?) -> ', lne)[0].split(' ')))

def get_selection_counts(s):
    return list(map(group_accents, re.findall('Counter\\((.+?)\\)', s)))+list(map(group_speakers, re.findall('Counter\\((.+?)\\)', s)))

def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9]+"run_{}/output/test_infer_log.txt".format(run_number)

def get_test(stats_file_path):
    return stats_file_path[:-9]+"run_1/output/test_infer_log.txt"

def WER_test_file(test_file):
    txt_file = open(test_file, 'r')
    lines = txt_file.readlines()
    matched = ""
    for line in lines:
        if "==========>>>>>>Evaluation Greedy WER: " in line:
            txt_file.close()
            return float(line.rstrip().split(": ")[1])
    txt_file.close()
    return ""

def get_eta(func, eta):
    return "-n:"+str(float(eta[4:]))

In [41]:
budget = 500
# target = 50
target = 10
# features = '39'
features = 'w2v2'
csv_name = "report_{}_{}_{}.csv".format(budget, target, features)

In [42]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/accent/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/random/run_1/output
# CMU_expts/speaker_without/ABA/manifests/TSS_output/within/budget_100/target_50/SKA/run_3

cols = ['speaker', 'ground', 'function', 'similarity', 'duration', 'samples', 
        'WER-r1', 'WER-r2', 'WER-r3', 'WER-mean', 'WER-var', 'accents_run1', 'accents_run2', 'accents_run3', 'speakers_run1', 'speakers_run2', 'speakers_run3']
df = pd.DataFrame(columns = cols)

speakers = [f.name for f in os.scandir('./') if f.is_dir() and f.name != '.ipynb_checkpoints' and f.name != 'reserved_TSS_output' and f.name in accent_map]


# CMU_expts/speaker_without/ABA/manifests/TSS_output/within/budget_100/target_50/SKA/run_3
for speaker in speakers:
    # if speaker not in ['ZHAA', 'MBMPS']:
    #     continue
    if not(pathlib.Path('./{}/manifests/TSS_output/'.format(speaker)).is_dir()):
        print("no results for speaker {}".format(speaker))
        continue
    if 'within' not in get_dirs('./{}/manifests/TSS_output/'.format(speaker)):
        print("no within results for {}".format(accent))
        continue
    if not(pathlib.Path('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)).is_dir()):
        print("no {} results for budget {} and target {}".format(speaker, budget, target))
        continue
    for function in get_dirs('./{}/manifests/TSS_output/within/budget_{}/target_{}/'.format(speaker, budget, target)):
        if 'deprecated' in function:
            continue
        stats_file_path='./{}/manifests/TSS_output/within/budget_{}/target_{}/{}/stats.txt'.format(speaker, budget, target, function)
        try:
            stats_file = open(stats_file_path, 'r')
        except:
            continue
        lines = stats_file.readlines()
        df_selections = get_selection_counts(lines[5])
        total_selections, total_durations, speakered_selections, speakered_durations = map(get_each_run, lines[:4])
        sample_frac = mean([x[0]/x[1] for x in zip(speakered_selections, total_selections)])
        sample_total = mean(total_selections)
        duration_frac = mean([x[0]/x[1] for x in zip(speakered_durations, total_durations)])
        duration_total = mean(total_durations)
        df_duration = "{:.2f}/{:.2f}".format(duration_total*duration_frac, duration_total)
        df_samples = "{:.2f}/{:.2f}".format(sample_total*sample_frac, sample_total)
        try:
            wers = [WER_test_file(get_test_file_from_stats_path(i, stats_file)) for i in range(1,4)]
            wers = [x for x in wers if type(x)==float or type(x)==int]
            df_wer_mean = round(mean(wers), 2)
            df_wer_var = round(variance(wers), 3)
        except:
            print("no WER's in file", get_test_file_from_stats_path(1, stats_file))
            wers = [0,0,0]
            df_wer_mean = 0
            df_wer_var = 999
        speaker_new=speaker+"[{}/{}]".format(composed_accent_map[speaker], gender_map[speaker])
        if function in accent_map:
            function = function+"[{}]".format(gender_map[function])
        # if function=='SKA':
        #     print("reached")
        df = df.append(dict(zip(cols, [speaker_new, "within", function, 
                        "NA", df_duration, df_samples]+ wers+[df_wer_mean, round(df_wer_var**0.5, 3)] + df_selections)), 
                       ignore_index=True)
        stats_file.close()

no THV results for budget 500 and target 10
no YKWK results for budget 500 and target 10
no ERMS results for budget 500 and target 10
no YDCK results for budget 500 and target 10
no TNI results for budget 500 and target 10
no YBAA results for budget 500 and target 10
no TXHC results for budget 500 and target 10
no ZHAA results for budget 500 and target 10
no LXC results for budget 500 and target 10
no MBMPS results for budget 500 and target 10
no RRBI results for budget 500 and target 10
no NCC results for budget 500 and target 10
no HKK results for budget 500 and target 10
no TLV results for budget 500 and target 10
no SVBI results for budget 500 and target 10
no WER's in file ./EBVS/manifests/TSS_output/within/budget_500/target_10/equal_random/run_1/output/test_infer_log.txt
no HQTV results for budget 500 and target 10
no SKA results for budget 500 and target 10


In [43]:
df = df.sort_values(by=['speaker', 'similarity', 'ground', 'function'], ascending=True, ignore_index=True)
display(df)

,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3
0,ABA[ARB/M],within,equal_random,NA,1795.35/1795.35,525.33/525.33,20.94,20.98,20.50,20.81,0.266,{'ARB': 527},{'ARB': 530},{'ARB': 519},"{'ARB': {'SKA': 189, 'ZHAA': 179, 'YBAA': 159}}","{'ARB': {'SKA': 188, 'ZHAA': 183, 'YBAA': 159}}","{'ARB': {'SKA': 186, 'ZHAA': 183, 'YBAA': 150}}"
1,ASI[HIN/M],within,equal_random,NA,1795.18/1795.18,580.33/580.33,13.84,13.33,13.33,13.50,0.295,{'HIN': 584},{'HIN': 581},{'HIN': 576},"{'HIN': {'SVBI': 222, 'TNI': 188, 'RRBI': 174}}","{'HIN': {'SVBI': 217, 'TNI': 190, 'RRBI': 174}}","{'HIN': {'SVBI': 215, 'TNI': 184, 'RRBI': 177}}"
2,BWC[CHN/M],within,equal_random,NA,1791.19/1791.19,484.67/484.67,28.75,27.93,29.20,28.63,0.644,{'CHN': 477},{'CHN': 495},{'CHN': 482},"{'CHN': {'NCC': 170, 'TXHC': 169, 'LXC': 138}}","{'CHN': {'TXHC': 177, 'NCC': 167, 'LXC': 151}}","{'CHN': {'TXHC': 171, 'NCC': 166, 'LXC': 145}}"
3,EBVS[ESP/M],within,equal_random,NA,1793.01/1793.01,472.67/472.67,0.00,0.00,0.00,0.00,31.607,{'ESP': 460},{'ESP': 473},{'ESP': 485},"{'ESP': {'NJS': 182, 'ERMS': 144, 'MBMPS': 134}}","{'ESP': {'NJS': 188, 'ERMS': 143, 'MBMPS': 142}}","{'ESP': {'NJS': 192, 'ERMS': 149, 'MBMPS': 144}}"
4,HJK[KOR/F],within,equal_random,NA,1795.31/1795.31,467.33/467.33,11.01,11.49,11.70,11.40,0.354,{'KOR': 470},{'KOR': 464},{'KOR': 468},"{'KOR': {'YKWK': 170, 'HKK': 155, 'YDCK': 145}}","{'KOR': {'YKWK': 165, 'HKK': 158, 'YDCK': 141}}","{'KOR': {'YKWK': 163, 'HKK': 158, 'YDCK': 147}}"
5,NJS[ESP/F],within,equal_random,NA,1788.44/1788.44,440.33/440.33,15.72,14.81,14.70,15.08,0.559,{'ESP': 438},{'ESP': 441},{'ESP': 442},"{'ESP': {'EBVS': 152, 'ERMS': 150, 'MBMPS': 136}}","{'ESP': {'EBVS': 157, 'ERMS': 146, 'MBMPS': 138}}","{'ESP': {'EBVS': 156, 'ERMS': 145, 'MBMPS': 141}}"
6,PNV[VTN/F],within,equal_random,NA,1793.34/1793.34,469.33/469.33,17.42,17.64,17.53,17.53,0.110,{'VTN': 471},{'VTN': 465},{'VTN': 472},"{'VTN': {'THV': 165, 'HQTV': 159, 'TLV': 147}}","{'VTN': {'THV': 165, 'HQTV': 160, 'TLV': 140}}","{'VTN': {'HQTV': 165, 'THV': 165, 'TLV': 142}}"


In [38]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# accented selection: 76 76 76 -> 76.00
# accented duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

#Evaluation Greedy WER: 16.19
df2 = pd.read_csv(csv_name)
print(csv_name)
df2=df2.append(df)
df2 = df2.sort_values(by=['speaker', 'ground', 'similarity', 'function'], ascending=True, ignore_index=True)
# df2.assign(accent = df['speaker'].split('[')[1].split('/')[0] + df['two']**2).sort_values('accent')
display(df2)


report_500_10_w2v2.csv


,speaker,ground,function,similarity,duration,samples,WER-r1,WER-r2,WER-r3,WER-mean,WER-var,accents_run1,accents_run2,accents_run3,speakers_run1,speakers_run2,speakers_run3
0,ABA[ARB/M],all,FL2MI-n:1.0,euclidean,557.02/1796.42,160.00/476.00,23.1,23.5,23.43,23.34,0.214,"{'KOR': 130, 'ARB': 160, 'ESP': 106, 'VTN': 40...","{'KOR': 130, 'ARB': 160, 'ESP': 106, 'VTN': 40...","{'KOR': 130, 'ARB': 160, 'ESP': 106, 'VTN': 40...","{'KOR': {'YDCK': 78, 'HJK': 51, 'HKK': 1}, 'AR...","{'KOR': {'YDCK': 78, 'HJK': 51, 'HKK': 1}, 'AR...","{'KOR': {'YDCK': 78, 'HJK': 51, 'HKK': 1}, 'AR..."
1,ABA[ARB/M],all,GCMI-n:1.0,euclidean,1025.12/1797.53,272.00/475.00,20.87,22.22,21.16,21.42,0.711,"{'ARB': 272, 'ESP': 76, 'KOR': 56, 'VTN': 37, ...","{'ARB': 272, 'ESP': 76, 'KOR': 56, 'VTN': 37, ...","{'ARB': 272, 'ESP': 76, 'KOR': 56, 'VTN': 37, ...","{'ARB': {'SKA': 96, 'YBAA': 88, 'ZHAA': 88}, '...","{'ARB': {'SKA': 96, 'YBAA': 88, 'ZHAA': 88}, '...","{'ARB': {'SKA': 96, 'YBAA': 88, 'ZHAA': 88}, '..."
2,ABA[ARB/M],all,LogDMI-n:1.0,euclidean,763.50/1799.48,210.00/480.00,23.17,23.57,23.65,23.46,0.257,"{'ARB': 210, 'ESP': 89, 'VTN': 62, 'HIN': 53, ...","{'ARB': 210, 'ESP': 89, 'VTN': 62, 'HIN': 53, ...","{'ARB': 210, 'ESP': 89, 'VTN': 62, 'HIN': 53, ...","{'ARB': {'YBAA': 104, 'SKA': 59, 'ZHAA': 47}, ...","{'ARB': {'YBAA': 104, 'SKA': 59, 'ZHAA': 47}, ...","{'ARB': {'YBAA': 104, 'SKA': 59, 'ZHAA': 47}, ..."
3,ABA[ARB/M],all,random,NaN,0.00/1798.50,0.00/502.00,22.7,23.03,23.21,22.98,0.259,"{'ARB': 83, 'HIN': 94, 'CHN': 88, 'ESP': 95, '...","{'ARB': 77, 'CHN': 89, 'KOR': 85, 'HIN': 87, '...","{'HIN': 80, 'ARB': 73, 'KOR': 91, 'CHN': 94, '...","{'ARB': {'SKA': 31, 'ZHAA': 28, 'YBAA': 24}, '...","{'ARB': {'YBAA': 34, 'SKA': 23, 'ZHAA': 20}, '...","{'HIN': {'ASI': 29, 'TNI': 18, 'SVBI': 18, 'RR..."
4,ASI[HIN/M],all,FL2MI-n:1.0,euclidean,1779.60/1798.23,513.00/520.00,13.51,14.37,13.19,13.69,0.610,"{'HIN': 513, 'ARB': 4, 'KOR': 1, 'VTN': 2}","{'HIN': 513, 'ARB': 4, 'KOR': 1, 'VTN': 2}","{'HIN': 513, 'ARB': 4, 'KOR': 1, 'VTN': 2}","{'HIN': {'RRBI': 367, 'SVBI': 106, 'TNI': 40},...","{'HIN': {'RRBI': 367, 'SVBI': 106, 'TNI': 40},...","{'HIN': {'RRBI': 367, 'SVBI': 106, 'TNI': 40},..."
5,ASI[HIN/M],all,GCMI-n:1.0,euclidean,1782.95/1797.33,506.00/511.00,12.62,13.44,13.76,13.27,0.588,"{'HIN': 506, 'VTN': 3, 'ARB': 2}","{'HIN': 506, 'VTN': 3, 'ARB': 2}","{'HIN': 506, 'VTN': 3, 'ARB': 2}","{'HIN': {'RRBI': 373, 'SVBI': 83, 'TNI': 50}, ...","{'HIN': {'RRBI': 373, 'SVBI': 83, 'TNI': 50}, ...","{'HIN': {'RRBI': 373, 'SVBI': 83, 'TNI': 50}, ..."
6,ASI[HIN/M],all,LogDMI-n:1.0,euclidean,1519.91/1797.94,483.00/581.00,12.58,12.62,12.54,12.58,0.045,"{'HIN': 483, 'ESP': 23, 'VTN': 35, 'KOR': 12, ...","{'HIN': 483, 'ESP': 23, 'VTN': 35, 'KOR': 12, ...","{'HIN': 483, 'ESP': 23, 'VTN': 35, 'KOR': 12, ...","{'HIN': {'RRBI': 234, 'TNI': 149, 'SVBI': 100}...","{'HIN': {'RRBI': 234, 'TNI': 149, 'SVBI': 100}...","{'HIN': {'RRBI': 234, 'TNI': 149, 'SVBI': 100}..."
7,ASI[HIN/M],all,random,NaN,0.00/1796.95,0.00/491.67,13.19,13.37,14.12,13.56,0.493,"{'HIN': 87, 'VTN': 100, 'KOR': 74, 'ESP': 81, ...","{'KOR': 105, 'HIN': 77, 'CHN': 68, 'ESP': 83, ...","{'VTN': 95, 'ARB': 90, 'HIN': 74, 'KOR': 87, '...","{'HIN': {'SVBI': 34, 'TNI': 31, 'RRBI': 22}, '...","{'KOR': {'YKWK': 29, 'YDCK': 27, 'HJK': 27, 'H...","{'VTN': {'PNV': 30, 'THV': 24, 'HQTV': 22, 'TL..."
8,BWC[CHN/M],all,FL2MI-n:1.0,euclidean,696.54/1796.97,181.00/460.00,29.24,29.73,29.09,29.35,0.335,"{'KOR': 200, 'CHN': 181, 'VTN': 77, 'ESP': 2}","{'KOR': 200, 'CHN': 181, 'VTN': 77, 'ESP': 2}","{'KOR': 200, 'CHN': 181, 'VTN': 77, 'ESP': 2}","{'KOR': {'YKWK': 127, 'HKK': 72, 'YDCK': 1}, '...","{'KOR': {'YKWK': 127, 'HKK': 72, 'YDCK': 1}, '...","{'KOR': {'YKWK': 127, 'HKK': 72, 'YDCK': 1}, '..."
9,BWC[CHN/M],all,GCMI-n:1.0,euclidean,707.55/1797.81,181.00/460.00,29.05,29.02,29.17,29.08,0.077,"{'KOR': 204, 'CHN': 181, 'VTN': 72, 'ESP': 3}","{'KOR': 204, 'CHN': 181, 'VTN': 72, 'ESP': 3}","{'KOR': 204, 'CHN': 181, 'VTN': 72, 'ESP': 3}","{'KOR': {

In [39]:

df2.to_csv(csv_name, index=False)
# input_file = csv.DictReader(open(csv_name))